In [5]:
import gym
import numpy as np
import time
import random
import operator
from IPython.display import clear_output
from gym import wrappers
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [6]:
env = gym.make("CartPole-v1")
#env = wrappers.Monitor(env, '/tmp/cartpole-experiment-5')
env.reset();

In [7]:
env.step(1)

(array([ 0.01874681,  0.19577089,  0.00111827, -0.32917018]), 1.0, False, {})

In [8]:
# for i in range(100):
#     env.step(random.randint(0,1))
#     env.render()
# env.reset()

In [9]:
pset = gp.PrimitiveSet("main", 4)
def if_then_else(input, output1, output2):
    return output1 if input else output2

pset.addPrimitive(if_then_else, 3)

pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)



pset.renameArguments(ARG0="x1")
pset.renameArguments(ARG1="y1")
pset.renameArguments(ARG2="x2")
pset.renameArguments(ARG3="y2")

In [10]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [11]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

In [17]:
def sample_reward(env, func, t_max=500):
    s = env.reset()
    total_reward = 0
    for _ in range(t_max):
        next_action = func(*s)
        next_action = 0 if next_action<-1 else 1
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        if is_done:
            break
    return total_reward

In [18]:
def evalSymbReg(individual, n_times=100):
    
    func = toolbox.compile(expr=individual)
    rewards = [sample_reward(env, func) for _ in range(n_times)]
    
    return float(np.mean(rewards)),

In [19]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [20]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [22]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)

   	      	            fitness            	              size             
   	      	-------------------------------	-------------------------------
gen	nevals	avg   	max 	min 	std    	avg  	max	min	std    
0  	100   	9.7434	13.7	9.15	1.01746	22.71	170	2  	34.0709
1  	56    	10.2409	13.53	9.19	1.35286	17.58	170	1  	25.2734
2  	44    	11.0483	13.53	9.21	1.63275	14.85	89 	2  	16.9949
3  	47    	11.8013	16.21	9.25	1.71712	14.65	89 	1  	15.7069
4  	49    	12.7967	18.86	9.24	1.55304	14.84	38 	2  	8.0271 
5  	49    	13.152 	18.86	9.22	2.02675	18.31	41 	2  	8.0953 
6  	63    	13.9068	39.25	9.23	3.45481	21.73	50 	4  	7.71732
7  	48    	15.7833	53.68	9.27	5.92632	22.91	50 	2  	9.1827 
8  	55    	18.3415	53.68	9.29	8.26718	22.67	47 	5  	8.87024
9  	43    	22.7069	54.7 	9.37	10.853 	22.62	46 	2  	9.59665
10 	46    	27.4763	111.73	9.31	16.0609	24.29	55 	3  	10.9529
11 	60    	39.2603	256.7 	9.29	33.6333	21.96	55 	2  	11.7872
12 	49    	56.6061	256.59	9.34	38.1574	17.72	52 	3  	8.57097
13 	52    	

KeyboardInterrupt: 

In [23]:
best_func = gp.compile(hof[0], pset)

In [24]:
evalSymbReg(hof[0])

(500.0,)

In [ ]:
for i in range(100):
    s = env.reset()
    total_reward = 0
    for _ in range(500):
        next_action = best_func(*s)
        next_action = 0 if next_action<=0 else 1
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        env.render()
        if is_done:
            break


TypeError: must be str, not bytes